In [43]:
!pip install keras-preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input
from keras.models import Sequential, Model
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [45]:
df = pd.read_csv('drive/MyDrive/CSC413_data/leetcode.csv')

df.sample(10)

,description,Array,Dynamic Programming,String,Math,Tree,Depth-first Search,Greedy,Hash Table,Binary Search,...,Random,Dequeue,Binary Search Tree,Suffix Array,Rolling Hash,Reservoir Sampling,Rejection Sampling,Memoization,OOP,Meet in the Middle
1322,"Given `numBottles` full water bottles, you can...",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
18,"Given the `head` of a linked list, remove the ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1552,"You are given three positive integers: `n`, `i...",0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
33,Given an array of integers `nums` sorted in as...,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1570,"You are given two integers, `m` and `k`, and a...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
816,"In an election, the `i`-th vote was cast for `...",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
725,We are given non-negative integers nums[i] whi...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1131,Given `head` which is a reference node to a si...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1342,Given a string `s`. An awesome substring is a ...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
437,You are given an array of binary strings `strs...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
inputs = df['description'].values
inputs = [i.lower() for i in inputs]
labels = df.iloc[:, 1:].values

print(inputs[0])
print(labels[0])

given an array of integers `nums` and an integer `target`, return indices of the two numbers such that they add up to `target`.

you may assume that each input would have exactly one solution, and you may not use the same element twice.

you can return the answer in any order.


example 1:
input: nums = [2,7,11,15], target = 9
output: [0,1]
output: because nums[0] + nums[1] == 9, we return [0, 1].

example 2:
input: nums = [3,2,4], target = 6
output: [1,2]

example 3:
input: nums = [3,3], target = 6
output: [0,1]

constraints:
`2 <= nums.length <= 103`
`-109 <= nums[i] <= 109`
`-109 <= target <= 109`
only one valid answer exists.
[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]


In [47]:
target_cols = df.iloc[:, 1:].columns.tolist()
print(target_cols)

['Array', 'Dynamic Programming', 'String', 'Math', 'Tree', 'Depth-first Search', 'Greedy', 'Hash Table', 'Binary Search', 'Breadth-first Search', 'Sort', 'Two Pointers', 'Backtracking', 'Stack', 'Design', 'Graph', 'Bit Manipulation', 'Heap', 'Linked List', 'Recursion', 'Union Find', 'Sliding Window', 'Trie', 'Divide and Conquer', 'Segment Tree', 'Ordered Map', 'Queue', 'Minimax', 'Geometry', 'Binary Indexed Tree', 'Brainteaser', 'Topological Sort', 'Line Sweep', 'Random', 'Dequeue', 'Binary Search Tree', 'Suffix Array', 'Rolling Hash', 'Reservoir Sampling', 'Rejection Sampling', 'Memoization', 'OOP', 'Meet in the Middle']


In [48]:
# Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(inputs)
sequences = tokenizer.texts_to_sequences(inputs)

In [49]:
print(sequences[1])
print(inputs[1])

[22, 20, 25, 58, 133, 114, 234, 531, 266, 58, 133, 328, 89, 1, 159, 20, 1700, 14, 648, 80, 9, 46, 6, 205, 81, 160, 7, 301, 289, 203, 1, 58, 141, 9, 23, 1, 91, 57, 7, 234, 78, 22, 115, 307, 1, 58, 141, 225, 61, 230, 69, 373, 312, 580, 1, 34, 4, 452, 10, 2, 11, 1946, 3, 13, 5, 2085, 16, 26, 13, 12, 27, 4, 51, 21, 5118, 5119, 5120, 10, 3, 11, 1946, 4, 2085, 4, 12, 4, 10, 5, 11, 1946, 45, 45, 45, 45, 45, 45, 45, 2085, 45, 45, 45, 45, 12, 51, 45, 45, 45, 4, 4, 4, 2, 37, 1, 34, 6, 81, 14, 46, 234, 78, 8, 14, 1, 105, 2, 73, 4, 56, 124, 45, 41, 8, 184, 24, 1, 78, 173, 7, 34, 24, 213, 61, 66, 373, 620]
you are given two non-empty linked lists representing two non-negative integers. the digits are stored in reverse order, and each of their nodes contains a single digit. add the two numbers and return the sum as a linked list.

you may assume the two numbers do not contain any leading zero, except the number 0 itself.


example 1:
input: l1 = [2,4,3], l2 = [5,6,4]
output: [7,0,8]
explanation: 342

In [50]:
max_len = max([len(s) for s in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len)

In [51]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_layer)
lstm_layer = LSTM(128)(embedding_layer)
output_layer = Dense(43, activation='sigmoid')(lstm_layer)
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [86]:
# split some for test 
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.1, random_state=42)

(158, 896)
(1413, 896)


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [53]:
def label_to_text(labels):
    return [target_cols[i] for i in range(len(labels)) if labels[i]]

In [55]:
from sklearn import metrics

In [87]:
print(X_test.shape)
print(X_train.shape)

(158, 896)
(1413, 896)


In [99]:
X_test1 = pad_sequences(tokenizer.texts_to_sequences(['Given a string s, find the length of the longest substring without repeating characters.']), maxlen=max_len)
print(type(X_test1))
y_pred1 = model.predict(X_test1)

print(y_pred1)

<class 'numpy.ndarray'>
1/1 [==============================] - 0s 81ms/step
[[0.09927627 0.12463806 0.4777806  0.06370531 0.06461447 0.05752271
  0.06866443 0.11527993 0.02918401 0.06636051 0.0381     0.04894514
  0.06328255 0.0735985  0.03377885 0.04435901 0.04696668 0.01798249
  0.0214706  0.01844105 0.02188412 0.0147563  0.01659179 0.01812822
  0.00997185 0.0076053  0.00611882 0.00449055 0.00809954 0.00392858
  0.0045638  0.00533345 0.00457201 0.00221565 0.00571381 0.00342663
  0.00465457 0.0017565  0.0033142  0.00085898 0.00246134 0.00075346
  0.00062566]]


In [97]:
print(X_test.shape)

(158, 896)


In [103]:
# test label count accuracy, select top number of labels from prediction based on original label count
test_pred = model.predict(X_test)
accuracies = []
for i, res in enumerate(test_pred):
    # if i == 5: break
    top_n = np.argsort(res)[-y_test[i].sum():][::-1]
    pred_labels = [0] * len(target_cols)
    for j in top_n:
        pred_labels[j] = 1
    # print(f"Original labels: {label_to_text(y_test[i])}")
    # print(f"Predicted labels: {label_to_text(pred_labels)}")
    accuracies.append(metrics.accuracy_score(y_test[i], pred_labels))
print(f'mean accuracy: {np.mean(accuracies)}')

5/5 [==============================] - 3s 516ms/step
mean accuracy: 0.9420076538121871
